#### Outline
1. 读入KML文件，存为点云
2. 根据距离建立初始类别
3. 根据方向，距离等聚类获得每一条路的归属车点集(对于公交车上下行信息可以用于简单判断)
4. 筛选剔除不合适的点，如0速度点
5. 分时间计算平均速度，并写入KML文件

In [1]:
import numpy as np
import pandas as pd
import json 
import re

#### Section 1

In [2]:
from sklearn.neighbors import KDTree
import os

In [3]:
FilePath='E:/WorkSpace/py/travel_flow/data/roadnet/excel_ver'

PointSet=[]
Cnt=0

for root, dirs, files in os.walk(FilePath):
    for file in files:
        df=pd.read_excel(os.path.join(root, file),encoding='utf-8')
        df=pd.DataFrame(df['ns1:coordinates'])
        df.columns=['xy']

        for i in range(0,len(df)):
            if not isinstance(df['xy'][i], float):
                strs=df['xy'][i].split(' ')
                Cnt=Cnt+1
                for pairs in strs:
                    coordinates=pairs.split(',')
                    PointSet.append({'road_id':Cnt, 'co_x':float(coordinates[0]), 'co_y':float(coordinates[1])})
PointSet=pd.DataFrame(PointSet)
PointCloud=np.array(PointSet.loc[:,['co_x','co_y']])
PointSet
tree = KDTree(PointCloud)

#### Section 2

In [4]:
FilePath='E:/WorkSpace/py/travel_flow/data/sample_data_week/bus/'
FileName='bus-20200309.csv'
df=pd.read_csv(FilePath+FileName,encoding='utf-8')
df.drop(['???"DEVICEINDEXCODE"','LINEINDEXCODE','DIRECTION','CURRENTSTATIONINDEXCODE'],axis=1,inplace=True)
df.head()

,LINEDIR,LONGITUDE,LATITUDE,GPSTIME,SPEED
0,下行,120.594386,28.021144,2020/3/9 0:00,0.0
1,上行,120.648602,28.011177,2020/3/9 0:00,35.0
2,下行,120.717183,28.008483,2020/3/9 0:00,29.6
3,上行,120.599269,28.016125,2020/3/9 0:00,0.0
4,下行,120.838413,27.906455,2020/3/9 0:00,0.0


In [132]:
Road_dic={}
for i in range(0,len(df)):
    #0的直接剔除了_______-还没考虑路口，红绿灯等
    if not df['SPEED'][i]:
        continue
    Velxy=np.array(df.iloc[[i],[1,2]])
    dist,raw_ind = tree.query(Velxy,k=1)
    index=PointSet.loc[int(raw_ind)]['road_id'] 
#     if(dist<threshold):
    VelInfo={'Dir':df['LINEDIR'][i],'Lon':df['LONGITUDE'][i],'Lat':df['LATITUDE'][i],\
                               'GPS':pd.Timestamp(df['GPSTIME'][i]),'Speed':df['SPEED'][i]}
    if not index in Road_dic:
        Road_dic[index]=pd.DataFrame([VelInfo])
    else:
        Road_dic[index]=Road_dic[index].append(VelInfo,ignore_index=True)

KeyboardInterrupt: 

#### Section 3

In [133]:
# 根据行进方向，剔除错误的点______尚未附加到正确的
for key in Road_dic.keys():
    df_=Road_dic[key]
    direction = df_['Dir'].value_counts().index[0]
    df_=df_[df_.isin([direction])]

#### Section 4

In [134]:
#写入KML文件
import dict2xml
from xml.dom.minidom import parseString

In [135]:
xml_str=''
RoadGroup=PointSet.groupby('road_id')

def wrap_mark(index,df):
    road_str=''
    groups=df.groupby(pd.Grouper(key='GPS',freq='T'))
    
    for time,group in groups:
        if (not group.empty):
            dic={}
            dic['name']=''
            dic['Style']={'LineStyle':{'color':'ffFFFF00'},'PolyStyle':{'fill':0}}
            Poly=RoadGroup.get_group(index)
            poly_str=''
            for idx in Poly.index:
                poly_str=poly_str+str(Poly['co_x'][idx])+','+str(Poly['co_y'][idx])+' '
            dic['LineString']={'coordinates':poly_str}
            dic['TimeStamp']={'when':str(time)}
            dic['description']=str(group['Speed'].mean())
            road_str=road_str+dict2xml.dict2xml(dic,wrap='Placemark',indent=" ")+'\n'
    return road_str

for key in Road_dic.keys():
    xml_str+=wrap_mark(key,Road_dic[key])

xml_raw = '<?xml version="1.0" encoding="utf-8" ?>\n<kml xmlns="http://www.opengis.net/kml/2.2">\n<Document id="root_doc">\n<Folder><name>roadnet</name>\n' +xml_str+'\n</Folder>\n</Document></kml>'

In [137]:
# 写入KML文件
filename = "test.kml"
f = open(filename, "w")
f.write(xml_raw)   
f.close()